In [8]:
import requests as req
import json
import urllib.parse
import pandas as pd
import ast
from bs4 import BeautifulSoup
from copy import copy
import os
import shutil
from zipfile import ZipFile
import re
import glob
import tabula
import PyPDF2
from pdf2image import convert_from_path
#import textract
import csv
from pytesseract import image_to_string
from PIL import Image, ImageDraw
import numpy as np
import math
from itertools import chain
import requests

In [2]:
def entitify(text):
    return urllib.parse.quote(text)

In [3]:
def fwrite(response, fname):
    file = open(fname, "w")

    file.write(response.text)

    file.close()

In [4]:
def demonstrate_json(res):
    print(json.dumps(res, indent=4, sort_keys=True, ensure_ascii = False))

In [9]:
class ClientMining:
    
    def __init__(self, headers = {'Accept': 'application/json, text/plain, */*',
                  'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' 
                   }, path = 'C:/Users/Александр/Desktop/Parser_dom_RF/'):
        #C:/Users/Александр/Desktop/Parser_dom_RF/
        #/Users/deniszagorodnev/Desktop/
        self.headers = headers
        self.path = path
    
    
    
    client_list = None
    client_page = None
    cardId = None
    banks = None
    beneficiaries = None
    documents = None
    reports = None
    rpd = None
    houses = None
    
    def get_client_list(self, q):
    
        headers = copy(self.headers)
        
        
        url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/search/interactive/composite?searchValue=' 
    
        q = entitify(q)
        
        url = url + q
        
        r = req.get(url, headers=headers)
        
        res = r.json()
        
        self.client_list = res
    
        
    
    def show_client_list(self):
        demonstrate_json(self.client_list)
        
    
    
    def get_client_page(self, TITLE, NAME):
        
        res = self.client_list
    
        url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%B5%D0%B4%D0%B8%D0%BD%D1%8B%D0%B9-%D1%80%D0%B5%D0%B5%D1%81%D1%82%D1%80-%D0%B7%D0%B0%D1%81%D1%82%D1%80%D0%BE%D0%B9%D1%89%D0%B8%D0%BA%D0%BE%D0%B2/'
    
        headers = copy(self.headers)
        
        header2 = 'title'
        
        header3 = 'searchValue'
    
        data = json.dumps(res, indent=4, sort_keys=True)
    
        df = pd.DataFrame(ast.literal_eval(data)['data'])
        
        
        row = df[(df[header2] == TITLE) & (df[header3] == NAME)]
        
        cardId = row['cardId'].iloc[0]
        
        self.cardId = copy(cardId)
        
        title = entitify(TITLE.lower())
    
        url = url + title + '/' + cardId
        
        r = req.get(url, headers=headers)
         
        self.client_page = r.text
        
        
    def show_client_page(self):
        print(self.client_page)
    
    
                    
                    
                    
    def get_proj_list(self, search):
        
    
        cardId = copy(self.cardId)
        
        headers = copy(self.headers)
        
        search = entitify(search)
        
        url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA-%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA?search='
        url1 = '&devId='
        url2 = '&status=buildingFlag'
    
        r = req.get(url + search + url1 + cardId + url2, headers=headers).text
        
        parsed_html = BeautifulSoup(r, 'html.parser')
        
        data = json.loads(parsed_html.find('script', id="__NEXT_DATA__", type='application/json').text)
        return data
    
    
    def parse_proj_list(self):
        
        proj_list = copy(self.proj_list)
        self.houses = proj_list['props']['initialProps']['pageProps']['houses']
    
    def parse_client_page(self):
        
        
        res = copy(self.client_page)
        
        parsed_html = BeautifulSoup(res, 'html.parser')
    
        #data = json.loads(parsed_html.find('script', id="__NEXT_DATA__", type='application/json').text)
        
        res = parsed_html.find('script', id="__NEXT_DATA__", type='application/json')

        data = json.loads(res.contents[0])
        

        self.banks = data["props"]['initialProps']["pageProps"]['banks']
    
        self.beneficiaries = data["props"]['initialProps']["pageProps"]['beneficiaries']
    
        self.documents = data["props"]['initialProps']["pageProps"]["documents"]
    
        self.reports = data["props"]['initialProps']["pageProps"]["documents"]['report']
    
        self.rpd = data["props"]['initialProps']["pageProps"]["documents"]['rpd']
        
        
    def get_doquments_archive(self, inn):
        
        cardId = copy(self.cardId)
    
        url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/erz/deverz/'+cardId+'/document/download?tab=projectDeclarations&tab=developerReport'
    
        with req.get(url, stream=True) as r:
            r.raise_for_status()
            with open(self.path + inn + '_' +  cardId + '.zip', 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): 
                    f.write(chunk)
                    
                    
    def get_client_page_by_inn(self, inn):
        
        self.get_client_list(inn)
        #print(self.client_list)
        self.cardId = self.client_list['data'][0]['cardId']
        
        TITLE = 'застройщик'
        
        url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%B5%D0%B4%D0%B8%D0%BD%D1%8B%D0%B9-%D1%80%D0%B5%D0%B5%D1%81%D1%82%D1%80-%D0%B7%D0%B0%D1%81%D1%82%D1%80%D0%BE%D0%B9%D1%89%D0%B8%D0%BA%D0%BE%D0%B2/'
    
        headers = {'Accept': 'application/json, text/plain, */*',
                  'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' 
                   }
        
        title = entitify(TITLE.lower())
        
        url = url + title + '/' + self.cardId
        
        r = req.get(url, headers=headers)
        self.client_page = r.text
    
    
    def unzipper(self, zippath, cond = False):
    
        #print(zippath[:-4])
        dirpath = zippath[:-4] + '_package'  #10
        #dirpath = zippath[:-15] + '_package'
        #dirpath = zippath[:-26] + '_package'
        
        if os.path.exists(dirpath) and os.path.isdir(dirpath):
            shutil.rmtree(dirpath)
            
        os.mkdir(dirpath)
    
        with ZipFile(zippath, 'r') as zipObj:
            
            if cond:        
                listOfFileNames = zipObj.namelist()
                for fileName in listOfFileNames:
                    if fileName.endswith('.csv'):
                        zipObj.extract(fileName, 'temp_csv')
            else:
                zipObj.extractall(path = dirpath)
    
    def get_declar_name(self, declar):

            pdfFileObj = open(declar,'rb')

            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

           # print(declar)

            pages = convert_from_path(declar, 0)


            page = pages[0]

            pname = 'C:/Users/Александр/Desktop/Parser_dom_RF/test.png'
            page.save(pname, 'JPEG')

            text = image_to_string(Image.open('C:/Users/Александр/Desktop/Parser_dom_RF/test.png'), lang = 'rus')

            text_file = open('res', "w")
            text_file.write(text)
            text_file.close()


            with open('res', "r") as f:
                new_text = f.read()
            f.close()

            os.remove('res')
            os.remove('C:/Users/Александр/Desktop/Parser_dom_RF/test.png')

            nt = new_text
            
            try:
                return re.findall('\d{2}.\d{2}.\d{4}(?=[^0-9])' + '(.*?)' + 'Дата первичного размещения:', re.sub('\n', '', nt))[0]
            except:
                return 'Error'
        
    def get_table_data(self, path):
        data = []
    
        with open(path) as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                for val in row.values():
                    data.append(val)
        data = str(data)
        data = re.sub('\]|\[|,|\'', '', data)
        data = data.replace("\\n", "")
        return data

    
    
    
    
    def get_archive_by_inn(self, inn):
    
        res = self.get_client_page_by_inn(inn)
        self.get_doquments_archive(inn)
        self.unzipper(self.path + inn + '_' + self.cardId + '.zip')
        
    df = pd.DataFrame(columns=['Застройщик', 'Дом', 'План стоимость', 'Площадь помещений', 'Учредители', 'Бенефициары', 
                               'Дата выдачи разрешения на строительство',
                              'Срок действия разрешения на строительство',
                              'Последнее продление', '20% реализации', '40% реализации', '60% реализации', '80% реализации', '100% реализации', 'Жилой дом', 'Земельные права', 'Эскроу','Привлеченные средства', '19.7.1.1.1.1', '19.7.1.1.1.2', '19.7.1.1.1.3', '19.7.1.1.2.1', '19.7.1.1.2.2', '19.7.1.1.2.3', '19.7.1.1.3.1',
 '19.7.1.1.3.2', '19.7.1.1.3.3', '19.7.2.1.1.1', '19.7.2.1.1.2', '19.7.2.1.1.3', '19.7.2.1.2.1', '19.7.2.1.2.2', '19.7.2.1.2.3',
 '19.7.2.1.3.1', '19.7.2.1.3.2', '19.7.2.1.3.3', '19.7.3.1.1.1', '19.7.3.1.1.2', '19.7.3.1.1.3', '19.7.3.1.2.1', '19.7.3.1.2.2',
 '19.7.3.1.2.3', '19.7.3.1.3.1', '19.7.3.1.3.2', '19.7.3.1.3.3', '19.6.1.4', '19.6.1.5', '19.6.1.6'])
            
        
        
    def append_new_line(self, developer, house, plan_cost, space, accept_date, accept_time, last_prolong, reals, ishouse, land_rights, escrow, attracted_sum, capital, infos_19):
        ben = []
        uch = []
        

        for elem in self.beneficiaries:
            if elem['founderType'] == 'Бенефициар':
                ben.append( str(elem['personFullName']) + ' ' + str(elem['founderProportion']))   
            elif  elem['founderType'] == 'Учредитель':
                try:
                    uch.append(str(elem['inn']) + ' ' +  str(elem['founderPercent']))
                    
                except:
                    uch.append(str(elem['devId']) + ' ' +  str(elem['founderPercent']))

        new_line = dict(chain.from_iterable(d.items() for d in ({'Застройщик': developer, 
                               'Дом': house, 
                               'План стоимость': plan_cost,
                               'Площадь помещений' : space,                     
                              'Учредители' : uch,
                               'Бенефициары' : ben,
                               'Дата выдачи разрешения на строительство' : accept_date,
                               'Срок действия разрешения на строительство' : accept_time,
                               'Последнее продление' : last_prolong,
                                '20% реализации' : reals[0],
                               '40% реализации' : reals[1],
                               '60% реализации' : reals[2],
                               '80% реализации' : reals[3],
                               '100% реализации' : reals[4],
                               'Жилой дом' : ishouse,
                               'Земельные права' : land_rights,
                               'Эскроу' : escrow,
                                'Привлеченные средства' : attracted_sum,
                                'Уставной капитал' : capital,
                              
                              
                              }, infos_19)))
        
        return self.df.append(new_line, ignore_index=True)
    
    
    
    def get_info(self, table, info = 'Основания по которому лицо входит в соответствии с законодательством Российской Федерации о защитеконкуренции в одну группу лиц с застройщиком:'):
        try:
            res = re.search(info + '(.*?)' + "([0-9][.][0-9])", table).group(1)
            res = re.sub(' ', '', res)
            return res
        except:
                return 0


    
    
    def get_info2(self, table, info):
        try:
            res = re.search(info + '(.*?)' + "г.", table).group(1)
            #res = re.sub(' ', '', res)
            return res
   
        except:
            return 0


    def get_info1(self, table, info):
        try:
            res = re.search(info + '(.*?)' + "руб.", table).group(1)
            res = re.sub(' ', '', res)
            return res
        except:
            return 0
        
      
    def get_sum(self, test):
            if 'Суммапривлеченныхсредств:' in test:
                try:
                    res = int(test[-(len(test) - len('Суммапривлеченныхсредств:')):])
                    return res
                except:
                    return '0'

            else:
                try:
                    int(test)
                    return test
                except:
                    return '0'

    def get_capital(self, test):
        if 'Размеруставногокапиталазастройщика' in test:
            try:
                res = int(re.sub('Размеруставногокапиталазастройщика', '', test))
                return res
            except:
                return test

        elif 'уставногокапиталазастройщика' in test:
            try:
                res = int(re.sub('уставногокапиталазастройщика', '', test))
                return res
            except:
                return test

        else:
            try:
                int(test)
                return test
            except:
                return '0' 
            
    def get_space(self, space):
        space = str(space)
        if 'Суммаобщейплощадивсехжилыхинежилыхпомещений:' in space:
                try:
                    space = int(space[-(len(space) - len('Суммаобщейплощадивсехжилыхинежилыхпомещений:')):])
                    return str(space)
                except:
                    return '0'

        else:
                try:
                    res = int(space)
                    return str(space)
                except:
                    return '0'    


    def get_info3(self, table, info):
        try:
            res = re.search(info + '(.*?)' + "([0-9][.][0-9][.][0-9])", table).group(1)
            res = re.sub(' ', '', res)
            return res[:10]
        except:
            return 0
        
        
    def get_info5(self, table, info):
        res = re.search(info + '(.*?)' + "([0-9][.])", table).group(1)
        res = re.sub(' ', '', res)
        return res

    def get_info6(self, table, info):
        res = re.findall(info + '(.*?)' + "г.", table)
        return res
   
    def is_house(self, table):
        try:
            res = re.search('Вид объекта капитального строительства:' + '(.*?)' + "([0-9][.][0-9])", table).group(1)

            res = re.sub(' ', '', res)

            return(res == 'Многоквартирныйдом')

        except:
            return False
        
        
    def get_land_rights(self, table):
        info = ''

        res = re.search("Дата окончания действия права застройщика на земельный участок:(.*?)Дата государственной регистрации изменений в договор", table).group(1)
        res = re.sub(' ', '', res)
        return res[:10]

    list_of_infos = ['Планируемая стоимость строительства:18.1.1строительства ', 
                 'Планируемый квартал и год выполнения этапа реализации проекта строительства:17.1.2',
                'Дата выдачи разрешения на строительство:',
                'Срок действия разрешения на строительство:',
                'Последняя дата продления срока действия разрешения на строительство:']
    
    def get_escrow(self, table):
    
        res1 = 0

        try:
            res1 = re.search("19.7.3.1.1.1(.*?)руб.", table).group(1)
            res1 = int(re.sub(' ', '', res1))
            #print(res1)
        except:
            res1 = 0

        res2 = 0

        try:
            res2 = re.search("19.7.3.1.2.1(.*?)руб.", table).group(1)
            res2 = int(re.sub(' ', '', res2))
            #print(res2)
        except:
            res2= 0
            

        return res1 + res2
    
    
    
    
    
    def get_19_infos(self, table, info1, info2):
        
        res  = re.search( '(?<=' + info1 + ')(.*)(?=' + info2 + ')', table).group(1)
        res = re.sub(' ', '', res)
        res = re.findall(r'\d+', res)[0]
        if res[0] == '0' and len(res) > 1:
            res = '0'

        if len(res) > 4:
            res = res[:-2] + '.' + res[-2:]

        return float(res)
    
    def process_archive(self, inn):
        list_of_19_7 = ['19.7.1.1.1.1', '19.7.1.1.1.2', '19.7.1.1.1.3', '19.7.1.1.2.1', '19.7.1.1.2.2', '19.7.1.1.2.3', '19.7.1.1.3.1',
     '19.7.1.1.3.2', '19.7.1.1.3.3', '19.7.2.1.1.1', '19.7.2.1.1.2', '19.7.2.1.1.3', '19.7.2.1.2.1', '19.7.2.1.2.2', '19.7.2.1.2.3',
     '19.7.2.1.3.1', '19.7.2.1.3.2', '19.7.2.1.3.3', '19.7.3.1.1.1', '19.7.3.1.1.2', '19.7.3.1.1.3', '19.7.3.1.2.1', '19.7.3.1.2.2',
     '19.7.3.1.2.3', '19.7.3.1.3.1', '19.7.3.1.3.2', '19.7.3.1.3.3', '19.6.1.4', '19.6.1.5', '19.6.1.6']
        
        
        infos_19_7 = [['19.7.1.1.1.1', '19.7.1.1.1.2'], ['19.7.1.1.1.2', '19.7.1.1.1.3'], ['19.7.1.1.1.3', '19.7.1.1.2'],
              ['19.7.1.1.2.1', '19.7.1.1.2.2'], ['19.7.1.1.2.2', '19.7.1.1.2.3'], ['19.7.1.1.2.3', '19.7.1.1.3'],
              ['19.7.1.1.3.1', '19.7.1.1.3.2'], ['19.7.1.1.3.2', '19.7.1.1.3.3'], ['19.7.1.1.3.3', '19.7.2'],
              
              ['19.7.2.1.1.1', '19.7.2.1.1.2'], ['19.7.2.1.1.2', '19.7.2.1.1.3'], ['19.7.2.1.1.3', '19.7.2.1.2'],
              ['19.7.2.1.2.1', '19.7.2.1.2.2'], ['19.7.2.1.2.2', '19.7.2.1.2.3'], ['19.7.2.1.2.3', '19.7.2.1.3'],
              ['19.7.2.1.3.1', '19.7.2.1.3.2'], ['19.7.2.1.3.2', '19.7.2.1.3.3'], ['19.7.2.1.3.3', '19.7.3'],
              
              ['19.7.3.1.1.1', '19.7.3.1.1.2'], ['19.7.3.1.1.2', '19.7.3.1.1.3'], ['19.7.3.1.1.3', '19.7.3.1.2'],
              ['19.7.3.1.2.1', '19.7.3.1.2.2'], ['19.7.3.1.2.2', '19.7.3.1.2.3'], ['19.7.3.1.2.3', '19.7.3.1.3'],
              ['19.7.3.1.3.1', '19.7.3.1.3.2'], ['19.7.3.1.3.2', '19.7.3.1.3.3'], ['19.7.3.1.3.3', '20'],
              ['19.6.1.4', '19.6.1.5'], ['19.6.1.5', '19.6.1.6'], ['19.6.1.6', '19.7']
              
              ]
        
        self.get_client_page_by_inn(inn)
        self.parse_client_page()
        declares = glob.glob(self.path  + self.cardId + '_package' + '/*_pd*.pdf')
        
        for declare in declares:
            #print(declare, self.df)
            #вынимаем название дома
            
            house = None
            house = self.get_declar_name(declare)
            #вынимаем csv
            tabula.io.convert_into(declare, "C:/Users/Александр/Desktop/Parser_dom_RF/" + self.cardId + '.csv', output_format="csv", pages='all')
            #обрабатываем csv
            
            data = self.get_table_data("C:/Users/Александр/Desktop/Parser_dom_RF/" + self.cardId + '.csv')

            
            plan_cost = self.get_info1(data, 'Планируемая стоимость строительства:18.1.1строительства ')
            
            if plan_cost == 0:
                plan_cost = self.get_info1(data, 'Планируемая стоимость строительства:')
                
            try:
                land_rights = self.get_land_rights(data)
            except:
                land_rights = 0
            
            try:
                attracted_sum = re.search('20.1.5' + '(.*?)' + "руб", data).group(1)
                attracted_sum = re.sub(' ', '', attracted_sum)
                attracted_sum = self.get_sum(attracted_sum)
            except:
                attracted_sum = '0'
            
            try:
                capital = re.search('21.1.1' + '(.*?)' + "руб", data).group(1)
                capital = re.sub(' ', '', capital)
                capital = self.get_capital(capital)
            except:
                capital = '0'
            
            try:
                space = re.search('9.3.3' + '(.*?)' + "м2", data).group(1)
                space = re.sub(' ', '', space)
                space = self.get_space(space)
            except:
                space = 'error'
            
            try:
                ID = self.get_info5(data, 'Идентификационный номер налогоплательщика:')
            except:
                ID = '0'
            
            infos_19 = []
            
            for i in range(len(list_of_19_7)):
                try:
                    infos_19.append({list_of_19_7[i]: self.get_19_infos(data, infos_19_7[i][0], infos_19_7[i][1])})
                except:
                    infos_19.append({list_of_19_7[i]: 123.})
                    
                
            
            infos_19 = dict(chain.from_iterable(d.items() for d in infos_19))
            
            
            escrow = self.get_escrow(data)
            
            accept_date = self.get_info3(data, 'Дата выдачи разрешения на строительство:')
            accept_time = self.get_info3(data, 'Срок действия разрешения на строительство:') 
            last_prolong = self.get_info3(data, 'Последняя дата продления срока действия разрешения на строительство:')
            
            #тут проблема
            reals = self.get_info6(data, ' Планируемый квартал и год выполнения этапа реализации проекта строительства:17.1.2 ')
            if len(reals) < 5:
                reals = self.get_info6(data, 'Планируемый квартал и год выполнения этапа реализации проекта строительства:')
            if len(reals) < 5:
                reals = ['0', '0', '0', '0', '0']
            for i in range(len(reals)):
                if len(reals[i]) >= 16:
                    reals[i] = reals[i][-16:]
            
            ishouse = self.is_house(data)
            
            #пишем данные в таблицу
            self.df = self.append_new_line(inn, house, plan_cost, space, accept_date, accept_time, last_prolong,
                                          reals, ishouse, land_rights, escrow, attracted_sum, capital, infos_19
                                          )
            
            #все подчищаем
            
            
            os.remove("C:/Users/Александр/Desktop/Parser_dom_RF/" + self.cardId + '.csv')
    
    
    def clean_all(self, inn):
        #print(self.cardId)
        os.remove(self.path + str(self.cardId) + '.zip')
        shutil.rmtree(self.path + str(self.cardId) + '_package')
    
          
    

In [19]:
def process_developers(list_of_inns):
    
        mining = ClientMining()
        cnt = 94
        for inn in list_of_inns:
            cnt += 1
            try:

                #получаем архив документов и распаковываем его
                mining.get_archive_by_inn(inn)

                #обрабатываем архив

                mining.process_archive(inn)
                print(cnt, ' ', inn)
                #удаляем все связанное с документом
#            mining.clean_all()
#            print(inn)    
            except:
                print('косяк:', cnt, ' ', inn)

        return mining.df

In [ ]:
inns = pd.read_excel('../Стройка.xlsx')

In [20]:
list_of_inns = []
with open('list_of_inns.txt', 'r') as f:
    #list_of_inns.append(f.read().split('\n\n')) Строка Дениса
    list_of_inns.append(f.read().split('\n'))
list_of_inns = list_of_inns[0]

In [21]:
print(list_of_inns)

['7604245976', '5038129524', '7811683992', '5040150192', '5257080286', '5405053909', '2225161032', '5410076789', '5410076740', '5410081620', '2224188899', '7810592380', '9721079797', '5406976145', '3327122570', '3327129061', '2127027670', '2540229261', '3525472577', '3525168231', '3662275073', '3662274986', '3662276905', '3662276856', '3662276888', '3662276870', '3662276895', '3662108298', '3664097595', '6027168241', '6167041823', '7816523223', '5406605937', '3328445662', '7104526898', '6686131767', '7107135363', '7100002910', '0264078271', '3662132710', '7717761040', '7718083824', '5321072550', '1435168723', '7447228150', '7722291048', '4629052057', '3849067610', '6731071470', '6452124443', '3702243063', '2724056185', '6730033514', '5018161287', '7604235865', '1660188671', '7727750873', '5408028770', '2543090547', '6501315838', '3625009637', '0245963224', '3811183671', '6670468444', '2540260141', '3662256507', '7805786527', '2502061378', '7806557590', '0278965428', '5260462253', '2310

In [22]:
data_res = process_developers(list_of_inns)

95   7604245976
96   5038129524
97   7811683992
98   5040150192
99   5257080286
100   5405053909
101   2225161032
102   5410076789
косяк: 103   5410076740
104   5410081620
косяк: 105   2224188899
106   7810592380
косяк: 107   9721079797
108   5406976145
109   3327122570
110   3327129061
111   2127027670
112   2540229261
113   3525472577
114   3525168231
115   3662275073
116   3662274986
117   3662276905
118   3662276856
119   3662276888
120   3662276870
121   3662276895
122   3662108298
123   3664097595
124   6027168241
125   6167041823
126   7816523223
127   5406605937
128   3328445662
129   7104526898
130   6686131767
131   7107135363
132   7100002910
133   0264078271
134   3662132710
косяк: 135   7717761040
136   7718083824
137   5321072550
138   1435168723
139   7447228150
140   7722291048
141   4629052057
142   3849067610
143   6731071470
144   6452124443
145   3702243063
146   2724056185
147   6730033514
148   5018161287
149   7604235865
150   1660188671
151   7727750873
152   54

561   1650355852
562   3906223185
косяк: 563   9703030573
564   7810781281
565   5402041158
566   5407472599
567   8601066900
568   6658524406
569   5406808983
570   6829088819
571   3525419541
572   8904043595
573   3317027134
574   6658543367
575   6829158110
576   6829092124
577   5834117950
578   6658532735
579   3665118880
косяк: 580   7708545163
581   7725790366
582   1901102641
583   5260367024
584   7802680723
585   7733298503
586   4401198235
587   2511101965
588   7702428657
589   5503133743
590   7202208741
косяк: 591   7708397268
косяк: 592   6161093783
593   2801091853
594   2540259322
595   2311317325
596   7730014425
597   6321342656
598   3811472070
599   1648029756
600   3525388710
601   2130223241
602   3128148408
603   0274903741
604   7708772293
605   6678109327
606   7706040110
косяк: 607   5407982536
608   7602153945
609   7325155225
610   2537115708
611   2543150919
612   4703179461
613   3328422640
614   2130200533
615   5902229253
616   5902048850
617   2465328

1031   2721181318
1032   9715313632
1033   3906959793
1034   5003126924
1035   7203480852
косяк: 1036   6317149441
1037   4501213695
1038   4501223005
1039   4703140305
1040   4725009167
1041   5835103598
1042   6161090616
1043   6950229826
1044   2901279464
1045   6950170435
1046   5501267639
1047   5610235203
1048   2460112123
1049   7735092843
1050   6215020181
1051   6215018425
1052   6215020174
1053   7813609810
1054   7725852742
1055   7841088140
1056   1655401319
1057   5029169930
1058   2901304022
1059   3906194537
1060   2926010105
1061   4028051443
1062   2901191298
1063   7727441970
1064   5029165879
1065   5029070497
1066   7703058776
1067   7203363500
1068   7203363490
1069   7203363588
косяк: 1070   7802495110
1071   5259138051
1072   2634108216
1073   6950239038
1074   7810342397
1075   2319062929
1076   5902197019
1077   5027112060
1078   3015113531
1079   7810757810
1080   3906315020
1081   2632114542
1082   7810799345
1083   9705115857
1084   6670494980
1085   9725020

In [ ]:
list_of_inns = list(inns['Названия строк'])

#сюды список ИНН
data_res = process_developers(list_of_inns)



In [ ]:
def ust(x):
    
    if 'Размерполность' in str(x):
        
        return x.split(':')[1]
    
    else:
        return x
    

    
data_res['Уставной капитал'] = data_res['Уставной капитал'].apply(lambda x: ust(x))

In [ ]:
#data_res['Уставной капитал']

In [ ]:
from tabula.io import convert_into

In [ ]:
data_res.to_csv('C:/Users/Александр/Desktop/Parser_dom_RF/res_new.csv', index = True)

In [ ]:
data_res.to_excel('C:/Users/Александр/Desktop/Parser_dom_RF/result_domrf.xlsx', index = True)

In [ ]:
from scipy import interpolate
import scipy

x = np.asarray([3, 4, 5])

y = np.asarray([0, 1, 2])

xx, yy = np.meshgrid(x, y)

z = np.asarray([[1, 7, 5], [3, 12, 6], [3, 6, 2]])

f = interpolate.interp2d(x, y, z)

In [ ]:
import unittest

class test_health(unittest.TestCase):
    
    def __init__(self, health_state = 'stable'):
        
        self.health_state = health_state

    def test_isill(self):
        
        self.assertTrue(self.health_state == 'ill')




In [ ]:
if __name__ == '__main__':
    unittest.main()

In [ ]:
from unittest import TestCase, main as unittest_main

class TestSimpleFoo(TestCase):
    foo = 'bar'

    def setUp(self):
        pass

    def test_a(self):
        self.assertEqual(self.__class__.foo, 'bar')
        self.__class__.foo = 'can'

    def test_f(self):
        self.assertEqual(self.__class__.foo, 'can')


if __name__ == '__main__':
    unittest_main()